# Web scraping-Ratings Prediction Project

In [1]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

import time, sys
import datetime
import tqdm.notebook as tqdm
import pandas as pd
import numpy as np

import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [2]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
#lets first connect to web driver
#driver=webdriver.Chrome('/Users/apple/Documents/drivers/chromedriver')
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/apple/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [4]:
driver = webdriver.Chrome(service=s)

# Reviews from Flipkart.com

In [5]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)
#clicking on cancel buttopn 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()
srch_items = ['laptops', 'Phones','smart watches','Monitors']
title = []
review_text = []
ratings = []

In [6]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return
urls=[]
for i in srch_items:
    # Find the search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
len(urls)            

384

In [7]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue
len(ratings), len(review_text), len(title)        

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=99.0.4844.51)
Stacktrace:
0   chromedriver                        0x0000000100b67159 chromedriver + 5120345
1   chromedriver                        0x0000000100af4b13 chromedriver + 4651795
2   chromedriver                        0x00000001006e4e68 chromedriver + 392808
3   chromedriver                        0x00000001006decf0 chromedriver + 367856
4   chromedriver                        0x00000001006d056d chromedriver + 308589
5   chromedriver                        0x00000001006d12f6 chromedriver + 312054
6   chromedriver                        0x00000001006d07d7 chromedriver + 309207
7   chromedriver                        0x00000001006cfd5b chromedriver + 306523
8   chromedriver                        0x00000001006cebf8 chromedriver + 302072
9   chromedriver                        0x00000001006cefe2 chromedriver + 303074
10  chromedriver                        0x00000001006e687a chro

In [ ]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
df

# Reviews from Amazon.in


In [8]:
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 
              'Monitors', 'Home theater', 'Router']
title = []
review_text = []
ratings = []

In [ ]:
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 
len(ratings), len(title), len(review_text)

In [ ]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
df